<a href="https://colab.research.google.com/github/rdemarqui/llm_complaint_management/blob/main/Tim%20Data%20Academy/tim_data_academy_mistral7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tagging Customer Feedback with LLM

This study aims to explore the advanced LLM tool Mistral 7B to analyze a vast collection of complaints gathered from the website [reclameaqui.com.br](reclameaqui.com.br). Through this analysis, we intend to demonstrate the potential and effectiveness of LLMs in interpreting and handling large-scale user feedback.

## Complaint Dataset

### Load

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

sns.set_theme()

In [ ]:
# Loading data
data_url = 'https://raw.githubusercontent.com/rdemarqui/sbrc_2024/main/data/df_validation.xlsx'
data = pd.read_excel(data_url, usecols=['problem', 'description'])
print(data.shape)

### Clean

In [ ]:
# Remove speciac characters, extra space and uncase all words
data['description'] = data['description'].astype(str)
data['description'] = data['description'].str.replace(r'<', ' ', regex=True)
data['description'] = data['description'].str.replace(r'>', ' ', regex=True)
data['description'] = data['description'].str.replace(r'#', ' ', regex=True)
data['description'] = data['description'].str.replace(r'\s{2,}', ' ', regex=True)
data['description'] = data['description'].str.lower()

In [ ]:
# Modify mask made by reclame aqui
def remove_mask(text):
  return text.replace("[editado pelo reclame aqui]", "[mask]")

data['description'] = data['description'].apply(remove_mask)

In [ ]:
# Remove description less than 3 characters
data = data[data['description'].str.len() > 3].copy()
df = data.copy()

### Analysis

In [ ]:
# Show each problem category
df['problem'].value_counts()

In [ ]:
# Show words statistics
df['word_count'] = df['description'].str.split().str.len()
df['text_len'] = df['description'].str.len()
df['word_count'].describe()

In [ ]:
# Check correlation between word count and text lenght
chart_joint = sns.jointplot(data=df, x="text_len", y="word_count", height=5)
chart_joint.fig.suptitle("Correlation between word count and text lenght")
chart_joint.fig.tight_layout()
chart_joint.fig.subplots_adjust(top=0.92)

In [ ]:
# Check word median per category
df.groupby('problem')['word_count'].median()

## Text Classification

### Load Model

There are several 7B models available as open source, such as Mistral, LLama, Falcon, Zephyr, and Openchat. In this study we will use Mistral, but other models can also be tested, just paying attention to adapting the instruction structure.

In [ ]:
import time
from datetime import timedelta, datetime
from google.colab import files
from IPython.display import clear_output

In [ ]:
%%capture
# Install vllm dependency
!pip install vllm

In [ ]:
from vllm import LLM, SamplingParams

We will load the pre-trained Mistral 7B model, quantized by the user TheBloke and available at: https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-AWQ

In [ ]:
# Load the LLM
model_id = "TheBloke/Mistral-7B-Instruct-v0.2-AWQ"
sampling_params = SamplingParams(temperature=0.0, top_p=1.0, max_tokens=60)
model = LLM(
    model=model_id,
    quantization='awq',
    dtype='half',
    gpu_memory_utilization=.95,
    max_model_len=4096
)

### Prompt Engineering

Prompt Guides:
* https://docs.mistral.ai/guides/prompting-capabilities/
* https://www.promptingguide.ai/models/mistral-7b
* https://huggingface.co/docs/transformers/main/tasks/prompting

In [ ]:
# Define tags
tags = ['sinal/conexão de rede', 'cobrança indevida', 'consumo saldo/crédito',
        'plano/benefício', 'cancelamento linha/plano', 'chip/sim card', 'spam',
        'portabilidade', 'recarga/pagamento', 'dificuldade de contato']

# Transform tag list in one string
tags_comma = ", ".join(f"{tag}" for tag in tags)

In [ ]:
# Examples
dialog_dict = {
    "conv1_1t": ["Esse plano é ruim!", "plano/benefício"],
    "conv2_1t": ["Recebo muitas mensagens de propaganda", "spam"]
}

In [ ]:
complaint_text = """
Reclamação: {user_complain}
"""

task = """
Tarefa: Classifique a reclamação. Atenção use apenas as categorias abaixo.
{tags}

Importante, apenas classifique sem explicar!
"""
answer = """
Rótulos:"""

#### Zero-shot

In [ ]:
zero_shot = ('[INST]' + complaint_text
             + task.format(tags=tags_comma)
             + answer + '[/INST]')
print(zero_shot)

#### Few-shot

In [ ]:
few_examples = """
###
Abaixo alguns exemplos:

Reclamação: {ex1}
Rótulos: {cat1}
Reclamação: {ex2}
Rótulos: {cat2}
###
""".format(ex1=dialog_dict['conv1_1t'][0], cat1=dialog_dict['conv1_1t'][1],
           ex2=dialog_dict['conv2_1t'][0], cat2=dialog_dict['conv2_1t'][1])

In [ ]:
few_shot = (
    '[INST]'
    + complaint_text + task.format(tags=tags_comma)
    + few_examples + answer
    + '[/INST]')

print(few_shot)

#### Multi-turn conversation

In [ ]:
conv = """<s>[INST]
Reclamação: {user_complain}

Tarefa: Classifique a reclamação. Atenção use apenas as categorias abaixo.
{tags}

Importante, apenas classifique sem explicar!

Rótulos:[/INST] {answer}
</s>
"""
conv_1 = conv.format(user_complain=dialog_dict['conv1_1t'][0],
                     tags=tags_comma,
                     answer=dialog_dict['conv1_1t'][1])
conv_2 = conv.format(user_complain=dialog_dict['conv2_1t'][0],
                     tags=tags_comma,
                     answer=dialog_dict['conv2_1t'][1])

In [ ]:
multi_turn = (conv_1 + conv_2 + zero_shot)
print(multi_turn)

#### Prompt test

In [ ]:
# Print each prompt (check)
prompt_dict = {'zero_shot': zero_shot, 'few_shot': few_shot, 'multi_turn': multi_turn}

In [ ]:
def llm_analysis(prompt, complain):
  response = model.generate(prompt.format(user_complain=complain), sampling_params, use_tqdm=False)
  llm_classification = response[0].outputs[0].text.replace('.', '').lower().split('\n')[0]

  return llm_classification

In [ ]:
def iterate_over_interval(df, column, prompt, limit_char, start_row, end_row, save):
  start_time = time.time()

  for nloop, index in enumerate(range(start_row, end_row)):
    complaint_content = str(df.loc[index, column]).lower()

    # Checkpoint
    if (nloop + 1) % 100 == 0:
      delta_time = str(timedelta(seconds=time.time() - start_time)).split('.')[0]
      print(f"{delta_time} Analysis number {nloop + 1}")

    # Limit text lengh
    if len(complaint_content) > limit_char: complaint_content = complaint_content[:limit_char]+'\n'

    #print(prompt.format(user_complain=complaint_content))  # check prompt

    # Text analysis
    analysis = llm_analysis(prompt, complaint_content)
    df.at[index, 'llm_class'] = analysis

  # Save final result
  delta_time = str(timedelta(seconds=time.time() - start_time)).split('.')[0]
  print(f"{nloop + 1} cases analyzed. Total execution time: {delta_time}")

  if save == True:
    current_datetime = datetime.now().strftime("%Y%m%d_%H%M%S")
    file_name = 'analise_reclamacao_' + str(start_row) + '_' + str(end_row) + current_datetime + '.xlsx'
    df.to_excel(file_name, index=False)

  return df, file_name if save == True else df

In [ ]:
# Run the example few times to see model consistency
example ="""como sempre, atendimento vago e sem nenhuma solução. estou a uma semana sem serviço na minha linha,
tentando falar na central e só fico presa na ura ou não consigo chegar a lugar algum. detalhe que tudo isso
precisa ser feito de outro celular já que não tenho sinal!!!!!!!!!!!!!não é a primeira vez que fico sem sinal
esse ano pelo chip virtual e que sou impactada em trabalho e já estou nisso a 1 semana. a resposta da central
foi: vá até uma loja física. eu tenho um chip virtual, qual o sentido de ir a uma loja física olhar um chip
virtual que o supervisor que consegui falar nem sabia do que se tratava???? fora que minha [ultima ida a loja
fisica fui cobrada por um chip dependente indevido que colocaram na minha conta e que nunca recebi reembolso.segue
protocolo:*******070quero saber qual o desconto que terei em fatura nesses dias sem serviço e não tenho
disponibilidade para ir a loja física por isso adquiri um chip virtual pela facilidade (que não existe do lado
de vocês) nesse processo.a qualidade do serviço é cada dia pior e eu ainda me pergunto pq sou cliente de vocës.
""".lower()

for prompt in prompt_dict:
  print('\n - ', prompt)
  for i in list(range(1, 4)):
    analysis = llm_analysis(prompt_dict[prompt], example)
    print(analysis)

### Multi-label Evaluation

In this section we will evaluate our prompt. To do that we labaled mannualy the **df_val** dataframe to use as comparision with model output.

There are a few metrics to evaluate multi-label classification, in this work we gonna use `classification_report`from `sklearn`, that is a well known package wich give us manny metrics to analyse. If you you want to know more about it, there's a good article made by Aniruddha Karajgi, on https://towardsdatascience.com/evaluating-multi-label-classifiers-a31be83da6ea.

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
# Creates binary classification columns and sum total classifications
def binary_cols(dataframe, column, tags):
  for tag in tags:
      dataframe[tag] = dataframe[column].apply(lambda x: 1 if tag in x else 0)

  dataframe[column + '_sum'] = dataframe[tags].sum(axis=1)

**Note:** Depending on the amount of data, the classification process can take a few hours. Using the quantized model described above, with zero shot prompt and utilizing the free tier infrastructure of Colab, I noticed that Mistral 7B takes about an hour to classify 2.000 cases.

Colab tends to disconnect due to inactivity. To prevent this, open your browser's inspector, go to the console tab, paste the code below, and press enter.

`function ClickConnect(){
    console.log("Working");
    document.querySelector("colab-connect-button").click()
}
setInterval(ClickConnect,60000)`

In [ ]:
# Open validation dataset tagged manually
df_val_url = 'https://raw.githubusercontent.com/rdemarqui/tim_data_academy/main/data/df_validation.xlsx'
df_val_labeled = pd.read_excel(df_val_url)

In [ ]:
# Check each prompt performance
y_expected = df_val_labeled[tags]
score_dict = {}
file_name_dict = {}

for prompt in prompt_dict:
  print("\n- ", prompt)
  temp_df, file_name = iterate_over_interval(df=df, column="description",
                                             prompt=prompt_dict[prompt],
                                             limit_char=2000, start_row=0,
                                             end_row=202, save=True)

  binary_cols(dataframe=temp_df, column='llm_class', tags=tags)

  y_pred = temp_df[tags]

  score_dict[prompt] = classification_report(y_expected, y_pred,
                                             output_dict=True,
                                             target_names=tags)
  file_name_dict[prompt] = file_name

There's some ways to agregate metrics such as `micro average`, `macro avarage`, `weighted average` and `samples average`. In this study we gonna use `samples avg`, wich was specifically designed for multilabel scenarios. It calculates metrics like precision, recall, and F1-score for each instance individually and averages them, thus effectively evaluating the model's performance on each sample by considering all its labels, making it particularly useful for assessing how well the model predicts the label set for each individual sample.

In [ ]:
# Create score comparison table
df_score_comparison = pd.DataFrame()

for prompt in prompt_dict:
  score_temp = score_dict[prompt]['samples avg']
  df_temp = pd.DataFrame(score_temp, index=[prompt])
  df_score_comparison = pd.concat([df_score_comparison, df_temp])

round(df_score_comparison, 4)

Remembering that:

* **Precision:** It is the proportion of positive identifications that were actually correct. A low precision indicates that there were **too many False Positives (FP)**, meaning that the model predicted many instances as positive that were actually negative.

 $\frac{\text{True Positive}}{\text{Total Predicted Positive}} = \frac{\text{True Positive (TP)}}{\text{True Positive (TP)} + \text{False Positive (FP)}}$

* **Recall:** It is the proportion of actual positives that were identified correctly. A low recall indicates that there were **too many False Negatives (FN)**, meaning that the model failed to identify many actual positive instances.

 $\frac{\text{True Positive}}{\text{Total Actual Positive}} = \frac{\text{True Positive (TP)}}{\text{True Positive (TP)} + \text{False Negative (FN)}}$

* **F1 Score:** Is the harmonic mean of the precision and recall, bringing a good balance between both.

 ${\text{F1}} = 2* \frac{\text{Precision * Recall}}{\text{Precision + Recall}}$

In this study we will use the prompt that achived the max F1 score, that represents a good balance between precision and recall.

## Results

In [ ]:
max_score_prompt = df_score_comparison['f1-score'].idxmax()
print(f"The max score obtained was achived by '{max_score_prompt}' prompt.")

In [ ]:
# Check detailed score from choosen prompt
round(pd.DataFrame(score_dict[max_score_prompt]).transpose(), 4)

---

In [ ]:
report = classification_report(y_expected, y_pred, target_names=tags)
print(report)